Connected to Python 3.13.7

In [2]:
from pyomo.environ import Set, Param, Var, NonNegativeReals, ConcreteModel, SolverFactory, Objective, Constraint, minimize, Binary, ConstraintList, NonNegativeIntegers
import pandas as pd
import numpy as np
import json
import random

# Get Data

In [3]:
DISTANCE_MATRIX_DIR = "../processed_data/distance_matrix_T2.csv"
DURATION_MATRIX_DIR = "../processed_data/duration_matrix_T2.csv"
SOME_STATIONS_DIR = "../processed_data/some_stations_T2.csv"

SOLUTION_DIR = "../results/solutionV2_test2_distance.csv"

distance_matrix = pd.read_csv(DISTANCE_MATRIX_DIR, header=None).to_numpy()
duration_matrix = pd.read_csv(DURATION_MATRIX_DIR, header=None).to_numpy()
some_stations = pd.read_csv(SOME_STATIONS_DIR, dtype={'station_id': str})

In [4]:
distance_matrix.shape

(41, 41)

In [5]:
duration_matrix.shape

(41, 41)

In [6]:
some_stations.head(20)

,station_name,station_id,latitude,longitude,net_flow,matrix_index
0,Pier 40 X2,SYS033,40.728487,-74.011693,0,0
1,Stagg St & Union Ave,5117.05,40.708771,-73.950953,-4,1
2,35 Ave & 10 St,6806.06,40.763155,-73.939855,-1,2
3,36 Ave & 10 St,6737.03,40.761438,-73.941126,-1,3
4,30 Ave & 96 St,6772.11,40.760860,-73.873080,0,4
5,14 St & 5 Ave,3771.06,40.666287,-73.988951,-2,5
6,Woodside Ave & 55 St,6276.02,40.746790,-73.908930,-7,6
7,47 Ave & Skillman Ave,6237.01,40.743752,-73.941765,7,7
8,Montague St & Clinton St,4677.06,40.694271,-73.992327,11,8
9,44 St & 48 Ave,6058.08,40.739414,-73.921320,-1,9


In [7]:
# filter out zero net_flow stations except for the warehouse
# Filter stations but PRESERVE original matrix indices
some_stations = some_stations[(some_stations['net_flow'] != 0) | (some_stations['station_name'] == 'Warehouse')]
# DON'T reset index here! Keep original indices

some_stations['station_name'] = some_stations['station_name'].str.strip()

print(f"Number of unique station_names with net flow != 0: {some_stations['station_name'].nunique()}")

Number of unique station_names with net flow != 0: 27


In [8]:
some_stations

,station_name,station_id,latitude,longitude,net_flow,matrix_index
1,Stagg St & Union Ave,5117.05,40.708771,-73.950953,-4,1
2,35 Ave & 10 St,6806.06,40.763155,-73.939855,-1,2
3,36 Ave & 10 St,6737.03,40.761438,-73.941126,-1,3
5,14 St & 5 Ave,3771.06,40.666287,-73.988951,-2,5
6,Woodside Ave & 55 St,6276.02,40.746790,-73.908930,-7,6
7,47 Ave & Skillman Ave,6237.01,40.743752,-73.941765,7,7
8,Montague St & Clinton St,4677.06,40.694271,-73.992327,11,8
9,44 St & 48 Ave,6058.08,40.739414,-73.921320,-1,9
10,Adelphi St & Myrtle Ave,4620.02,40.693083,-73.971789,-5,10
13,E 33 St & 5 Ave,6322.01,40.747659,-73.984907,12,13


In [9]:
print(f"Sum of all flows: {sum(some_stations['net_flow'])}")

Sum of all flows: 0


<div>
<center>
<img src="https://cdn.vox-cdn.com/thumbor/RAf_4Lk9jN8WhU5RO1J8M4iMQU8=/1400x1400/filters:format(jpeg)/cdn.vox-cdn.com/uploads/chorus_asset/file/19211000/916126092.jpg.jpg" width="500"/>
</center>
</div>

# <center> New York Citi Bike Problem </center>

As you may be aware, the growth of the shared E-Bike industry is growing fast. By the end of 2030 the industry is expected to grow by a whopping 11.3% compared to 2025. In August of 2025, _New York City_ broke the world record for the **most** number of bicycle trips in a single day; **205,000 total trips**.

The main operator in *New York City* is a private company called *Citi Bike* with a **33,000** large fleet of bikes (including E-Bikes). The winner of the most recent *2025 New York Mayoral Election*, *Zohran Mamdani*, an active Citi Bike user who campaigned to sponsor companies like *Citi Bike*, has contacted us with a job ...

*New York City* is considered the world center for stress due to long working hours, expensive prices and bad transportation. This is all the more felt during the *morning rush* where stock brockers need to get to trading firms, doctors to hospitals and pizza makers to their ovens. The ensure everyone can get to their destination, Zohran Mamdani has asked us to help solve a key problem with *Citi Bikes* , and other bike sharing systems: 

Most of the time, when you **need** a bike, there are none available, and when you **have** a bike, you have nowhere to park them because the docks are full. This is a common issue with bike sharing systems and to solve this, employees drive with buses which balance the number of bikes across the system by ensuring overloaded stations offload some bikes to the stations lacking bikes. But because its *rush hour*, Zohran Mamdani wants this done **as fast as possible**. If the system is proven to work, he might deploy it for the *entire day* with the criteria of being **as efficient as possible**

# <center> The Math </center>
This problem is referred to as a Network Problem. In our case, we are modelling it as a **Linear Discrete Mixed-Integer Split Delivery and Pick-up Vehicle Routing Problem**. A lot of words, but by the end of this notebook all the decisions made should be clear and hopefully justified by the solution.

A network is just as it sounds, a grouping of nodes (**Verticies** $V$) joined together by lines (**Edges** $E$). For a given *vertex*, we connect it to **every other** *vertex* except itself, in such a way that the number of **non-unique** edges becomes:
$$ E = (V-1)V $$

In our case, we can consider a *vertex* as a station in the bike-sharing system such that the *edges* are simply connections between stations which symbolize a path from one station to another.

## Linear Modelling
For us to solve this problem, we want to use linear programming. A linear programming problem consists of the following:
- **Sets** 
- **Parameters**
- **Decision Variables**
- **Constraints**
- **Objective Function**

### Sets
The sets designed for this problem are the foundations for the rest of the problem, so getting these right was very important.

Suppose *Citi Bike* has some $k$ trucks available for balancing the bike network, quite simply the set of all buses would be:
$$ \mathcal{K} = \set{\text{bus}_1, \text{bus}_2, \text{bus}_3, ... , \text{bus}_k } $$

As previosly mentioned, the number of *consumer vertices* that we are considering $V$ in this problem can also be modelled as a set: 
$$ \mathcal{S} = \set{\text{station}_1, \text{station}_2, \text{station}_3, \ldots , \text{station}_V} $$

Notice we mentioned *consumer vertices*. Well we must consider the start location of all of our buses, which is located at the **Warehouse** in the *Empire State Building*. As such the **set** of *all vertices*
$$ 
\begin{align*} 
\mathcal{N} &= \set{\text{station}_1, \text{station}_2, \ldots, \text{station}_V, \text{Warehouse}} \\
            &= \mathcal{S} \cup \set{\text{Warehouse}} 
\end{align*} 
$$

Almost there, now we can consider two *subsets* of our *consumer nodes*.
The first is the **set** of *supply nodes* or *pickups* given as:
$$ i \in \mathcal{J} \subset \mathcal{S} \quad \text{where} \quad b_i > 0 $$

Finally the last *subset* is the **set** of *demand vertices* or *dropoffs* given by:
$$ i \in \mathcal{D} \subset \mathcal{S} \quad \text{where} \quad b_i < 0 $$

To follow with the nomenclature of the sets, I will refer to *verticies* as **nodes**

### Parameters
Parameters are basically just numbers that describe elements in a give *set*. For example, for every human in the world (the *set* of all humans) we have parameters like height, weight, sex etc.

In our case, the parameters we care about are the following:
* $t_{ij}$, $d_{ij}$ | **Travel Cost (Time or Distance)** from node $i$ to node $j$. Note direction is important as real-world urban road networks are not bi-directional in most cases

* $b_i$ | **Net Flow** of bikes leaving (-) or entering (+) a given node $i$ 

* $B_i = |b_i|$ | **Required Service** for a given node. Basically a measure of how stuff needs to done at a give node $i$ 

* $Q$ | **Truck Capacity** for a given truck $k$.

### Decision Variables
Decision variables are the key to solving a linear programming problem. They are what your algorithm gets to decide. In primary high school science experiment terms, its the independant variable or the *thing you can change*.

In the case of the *Citi Bike* problem, our drivers have a limited form of free will. They *may* drive their truck $k$ from a given **start** node $i$ to an **end** node $j$, however they *cannot* stop along the way and **must** return to the depot at the end of their journey. What we are modelling is effectively a switch on if we want to activate a given *edge* $E$ or not. This behaviour can be modelled as:
$$ x_{ijk} \in \mathbb{B} \quad\forall i,j \in \mathcal{N},i\neq j, \forall k \in \mathcal{K} $$

The drivers will be driving trucks with a *limited* storage capacity $Q_k$. Which means we will need to track how many bikes are in those trucks throughout their journey. This will be useful later for the *constraints* incase we need to punish the drivers for going over the storage limits. 

The **load** (number of bikes) on a given truck $k$ **upon arrival** to node $i$ can be modelled as:
$$ y_{ik} \in \mathbb{R_0^+} \quad\forall i \in \mathcal{N}, \forall k \in \mathcal{K} $$

Similarly, we need to keep track of the actions of our drivers. We want to know the amount of bikes **serviced**(picked up or dropped off) by a given truck $k$ at a node $i$
$$ z_{ik} \in \mathbb{R_0^+} \quad\forall i \in \mathcal{N}, \forall k \in \mathcal{K} $$

Because the objective of our operation may change, we must consider a scenario where more trucks is actually worse than fewer trucks despite having a supply of $\mathcal{k}$ trucks. As such, we need a switch that tells us if a bus $k$ is being used or not:
$$ w_{k} \in \mathbb{B} \quad \forall k \in \mathcal{K}$$

Finally, because of a massive problem we might encounter in the future, we need this extra **auxillary** decision variable for each node $i$ and bus $k$:
$$ u_{ik} \in \mathbb{Z_0^+} \quad \forall i \in \mathcal{N} \quad \forall k \in \mathcal{K}$$


### Constraints
Most definetely the hardest part of the problem, the constraints. Getting this to work not only took ages, but can be un-intuitive. Adding constraints is benefitial to our computers, it limits the search problem down to fewer possible answers. But as humans, we like to over complicate problems (us) and make the search spaces larger. None the less, these are the constraints for our problem
* **Routing Constraints** 
* **Split Service & Flow Balance Constraints**
* **Load Tracking and Capacity Constraints** 
* **Subtour Elimination Constraints**

#### <u> Routing Constraints </u>
Routing constraints dictate where the trucks start, end and their natural behaviour
1. **Truck Departure from Warehouse (W)**
    $$ \sum_{j\in\mathcal{S}} x_{Wjk} = w_{k} \quad \forall k \in \mathcal{K} $$
    If the bus is used ($w_k = 1$) the bus must *leave from* the warehouse to *exactly* one destination
    
2. **Truck Arrival to Warehouse (W)**
    $$ \sum_{j\in\mathcal{S}} x_{jWk} = w_{k} \quad \forall k \in \mathcal{K} $$
    If the bus is used ($w_k = 1$) the bus must *return to* the warehouse from *exactly* one departure
3. **Flow Conservation**: If truck $k$ enters node $j$, it must leave node $j$.
    $$\sum_{i \in \mathcal{N}} x_{ijk} = \sum_{i \in \mathcal{N}} x_{jik} \quad \forall j \in \mathcal{S}, \quad \forall k \in \mathcal{K}$$

#### <u>Split Service & Flow Balance Constraints</u>
4. **Service Demand Fulfillment (Split Delivery)**: The total service amount (pickup or delivery) provided by all trucks at station $i$ must exactly meet the required imbalance $B_i$.
    $$\sum_{k \in \mathcal{K}} z_{ik} = B_i \quad \forall i \in \mathcal{S}$$

5. **Linking Routing to Service** : A truck $k$ can only service station $i$ if it visits station $i$.
    $$\sum_{j \in \mathcal{N}} x_{jik} \cdot M \geq z_{ik} \quad \forall i \in \mathcal{S}, \forall k \in \mathcal{K}, M = Q_k \cdot 2 $$
    If a truck does not visit a node $i$ such that $\sum_{j \in \mathcal{N}} x_{jik} = 0$ then $z_{ik} \leq 0$, thus the bus **cannot** service the node

6. **Service Limit**: The service amount $z_{ik}$ cannot exceed the required service amount $B_i$.
    $$z_{ik} \leq B_i \quad \forall i \in \mathcal{S}, \forall k \in \mathcal{K}$$
    We cannot drop off more bikes than are needed nor can we pick up more bikes than available. *ASSUMPTIONS*

#### <u> Load Tracking and Capacity Constraints </u>
7. **Truck Load at Depot**: All trucks start and end empty at the depot.
    $$y_{0k} = 0 \quad \forall k \in \mathcal{K}$$

8. **Load Capacity**: The load on the truck upon arrival at any station must not exceed the truck capacity
    $$y_{ik} \leq Q_k \quad \forall i \in \mathcal{N}, \forall k \in \mathcal{K}$$

9. **Load Flow Conservation** : This tracks the change in load on truck $k$ as it moves from node $i$ to node $j$.
The service operation (pickup or delivery) occurs at node $i$ before the truck proceeds to node $j$
    $$\text{Service}(i, k) = \begin{cases} z_{ik} & \text{if } i \in \mathcal{J} \text{ (Pickup)} \\ -z_{ik} & \text{if } i \in \mathcal{D} \text{ (Delivery)} \\ 0 & \text{if } i = 0 \text{ (Depot)} \end{cases}$$
    **<center> Lower Bound** </center>

    $$y_{ik} + \text{Service}(i,k) - y_{jk} \leq M(1 - x_{ijk}) \quad \forall i \in \mathcal{S}, j \in \mathcal{N}, i \neq j, \forall k \in \mathcal{K}$$

    **<center> Upper Bound** </center>

    $$y_{ik} + \text{Service}(i,k) - y_{jk} \geq -M(1 - x_{ijk}) \quad \forall i \in \mathcal{S}, j \in \mathcal{N}, i \neq j, \forall k \in \mathcal{K}$$

11. **Non-Negative Load**: The load on the truck must never be negative.  
    $$y_{ik} \geq 0 \quad \forall i \in \mathcal{N}, \forall k \in \mathcal{K}$$

12. **Load Feasibility Check** (Crucial for Pickups/Deliveries):  
    a. Pickups ($i \in \mathcal{J}$):  
        Load after pickup ($y_{ik} + z_{ik}$) must not exceed $Q$.  
        $$y_{ik} + z_{ik} \leq Q + M(1 - \sum_{j \in \mathcal{N}} x_{jik}) \quad \forall i \in \mathcal{V}, \forall k \in \mathcal{K}$$

    b. Deliveries ($i \in \mathcal{D}$):  
        Load before delivery ($y_{ik}$) must be sufficient to cover the delivery amount ($z_{ik}$).  
        $$y_{ik} - z_{ik} \geq - M(1 - \sum_{j \in \mathcal{N}} x_{jik}) \quad \forall i \in \mathcal{V}, \forall k \in \mathcal{K}$$

    Please refer to the bottom of the report for an explination

#### <u> Subtour Elimination Constraints (MTZ) </u>
What is the difficult part of a routing problem: **SUBTOURS**. This is by far the part of the project we spent most trying to fix. There are many types of Subroute Elimination Methods but they all have their pros and cons. Initially we tried using a **General Miller-Tucker-Zemlin** or **MTZ** on the path decision variable ($x_{ijk}$). This method is actually iterative and allows a more relaxed problem to be solved, it then takes all the subroutes (not including the Warehouse) and generates pair-wise constraints such to 'cut' the subroute open. This means, in theory, if the problem was fully constrained, a 100 node problem would generate $2^{100} -2$ constraints. This is just not feasible. So, with the help of 'Integer Programming Formulation of Traveling Salesman Problems' by C. E. Miller, A. W. Tucker, and R. A. Zemlin, the *auxilliary* variable was introduced.

13. **MTZ Initialization**: The depot is the origin/end of the route.
$$u_{0k} = 0 \quad \forall k \in \mathcal{K}$$

14. **MTZ Flow Linkage**: The auxiliary variable $u_{ik}$ increases along the path for a truck $k$. This prevents closed loops that do not include the depot.
    $$u_{ik} - u_{jk} + |\mathcal{S}| \cdot x_{ijk} \leq |\mathcal{S}| - 1 \quad \forall i, j \in \mathcal{S}, i \neq j, \forall k \in \mathcal{K}$$

15. **MTZ Bounds**: The value of $u_{ik}$ should be within the range of possible sequence positions.
    $$1 \leq u_{ik} \leq |\mathcal{S}| \quad \forall i \in \mathcal{S}, \forall k \in \mathcal{K}$$

    Please refer bellow for explinations.


### Objective Functions
Finally, the last step is to design the objective functions for our model. Basically the metric by which we judge our solutions for optimality.
* `As fast as possible`:
    We need to optimize for **minimum time**. This objective is not as simple as it seems. With two *distinct* buses, if we run then both from 7AM to 8AM then the total time is **not** 2 hours but 1 hour. To model this generally, together with a few assumptions, we can optimize for *minimum maximum time* between the buses. To do this we must introduce a last decision variable and a contraint:
    $$ \text{Let } T_{max} \in \mathbb{R^+_0} \quad \rightarrow \quad T_{max} \geq\sum_{i \in \mathcal{N}} \sum_{j \in \mathcal{N}, i \neq j} t_{ij} x_{ijk} \quad \forall k \in \mathcal{K}$$
    $$ \min \quad T_{max} $$

    The constraint acts as a lower bound (representing the longest duration out of all the buses) and the objective function is a soft constrint forcing $T_{max}$ to equal to longest duration bus.
* `As efficiently as possible`:
    Being a politican, the mayor naturally was quite vague with this constraint. As STEM people this can mean a lot, but given our shortage of data, we will assume it means that we optimize for **minimum total distance**:
    $$ \min \sum_{k \in \mathcal{K}} \sum_{i \in \mathcal{N}} \sum_{j \in \mathcal{N}, i \neq j} d_{ij} x_{ijk} $$

Finally our model is complete, all that is left is to get the data, code the model, run the model and then wait for an eternity for our Mixed-Integer Problem to be solved. Although New Yorkers are quite cool, they cannot ride half a bike, hence, our job is to move *integer* values of bikes. This is a drawback on our computational time because the computer will need Branch and Bound (a relatively fast algorithm) to find *a* solution to our problem. Thats right, we are guaranteed a problem, but if the search space is extremely large, by the time our computers find the optimal solution, we are over the deadline. There is not much around this as the nature of the Branch and Bound algorthm is that it explores all possible paths (with some extra bounds along to way to speed it up). Hence, by running it on our civilian laptops, we are not likely to find a solution within a reasonable time. However, as you will see, we have included two tests of different size such to show the degree to which using Linear Programming sucks at solving this problem.


## Key Assumptions
* **Closed System**: By considering an arbitrary number of stations/verticies, we must ensure that the *number of bikes in the systems is fixed*, ie. we only consider the flow of bikes between vertecies *within our network*:
$$ \sum_{i \in \mathcal{S}} b_i = 0 $$
* **Intermediate Loading/Off-Loading** : We assume that the trucks cannot offload or load bikes at a given node with the intention of returning. In other words, a truck wont offload more bikes at a given station with the intention of using it as a 'mini-supply hub' for that neighbourhood. This is quite realistic because dropping bikes off at a station that cannot support more bikes means those bikes are not being used/charged and at risk of being stolen and/or damages
* **Simoultaneous Operation**: We can assume that the trucks run simoultaneously. In other words by starting all the trucks at the same time, we can assume that the `minimum time` for the job is calculated for the bus that takes the longest.

#### <u> Where can we get out data from ? </u>
Luckily, Citi Bike publishes all their ride data for every day of the year. It contains features such as `start_station_name` ,`end_station_name`, `start_time`, `end_time`,`start_lat`,`end_long` etc. We can use this data to build a net flow matrix which will then feed the flow parameters of our model.
The first thing we did is collect the data, which is stored in `in_data` and contains all the data for November 2025. We then filter this data by working days `M,T,W,T,F` and by morning rush hour `7AM - 9AM`. With this data, we sample `N_Nodes-1` **unique** nodes. We collect all the data entries where `start_station_name` and `end_station_name` are match these unique nodes this way we have a closed system of nodes. Then we count the number of bikes flowing between each node and sum them where -1 is for outgoing and +1 is for ingoing. With the net flow matrix computed, we collect all the `(lat,long)` coordinates of each node. 

At this point we move to `https://project-osrm.org/` which is an open source mapping project with a great API call. With a single API call you can collect a distance and time matrix for up to a `100` $\times$ `100` matrix by simply feeding the call the `(lat,long)` coordinates of a given point. With this done, we save the data in `processed_data` for use.

# Build Model

In [10]:
model = ConcreteModel()
solver = SolverFactory('glpk')

N_BUSES = 4
MIN_BUS_CAPACITY = 10
MAX_BUS_CAPACITY = 50

DEPOT_INDEX = some_stations.index[some_stations['station_name'] == 'Warehouse'].tolist()[0]
print(f"Depot index: {DEPOT_INDEX}")

M = MAX_BUS_CAPACITY * 2  # A large constant for Big-M method
print(some_stations.shape)

def clear_model(model):
    model.clear()

Depot index: 40
(27, 6)


# SETS

In [11]:
clear_model(model)
def model_sets(model):
      model.nodes = Set(initialize=some_stations['station_name'].tolist())
      print(f"Total nodes: {model.nodes.__len__()}")

      model.supply_nodes = Set(initialize=
                              some_stations[some_stations['net_flow'] > 0]['station_name'])

      model.demand_nodes = Set(initialize=
                              some_stations[some_stations['net_flow'] < 0]['station_name'])

      model.consumer_nodes = model.supply_nodes | model.demand_nodes

      print(f"Supply nodes: {model.supply_nodes.__len__()}"
            f"\nDemand nodes: {model.demand_nodes.__len__()}"
            f"\nConsumer nodes: {model.consumer_nodes.__len__()}")

      model.trucks = Set(initialize=range(N_BUSES))


# PARAMETERS

In [12]:
def model_params(model):

    station_to_matrix_idx = dict(zip(some_stations['station_name'], some_stations['matrix_index']))

    # In your solver after loading the CSV:
    station_to_matrix_idx = dict(zip(some_stations['station_name'], some_stations['matrix_index']))

    # This should look up distance_matrix[9, 2]
    distance = distance_matrix[
        station_to_matrix_idx['Warehouse'],      # = 9
        station_to_matrix_idx['35 Ave & 10 St']  # = 2
    ]
    print(f"Distance from Warehouse to 35 Ave & 10 St: {distance:.4f} km")

    model.distance_param = Param(model.nodes, model.nodes,
        initialize=lambda model, i, j:
            distance_matrix[
                station_to_matrix_idx[i],  # Use matrix_index, not DataFrame index
                station_to_matrix_idx[j]
            ]
        )
    
    model.duration_param = Param(model.nodes, model.nodes,
        initialize=lambda model, i, j:
            duration_matrix[
                station_to_matrix_idx[i],  # Use matrix_index, not DataFrame index
                station_to_matrix_idx[j]
            ]
        )
    
    model.net_flow_param = Param(model.nodes,
                                    initialize=lambda model, i:
                                        some_stations[
                                            some_stations['station_name'] == i
                                        ]['net_flow'].values[0]
                                        )

    # B = |b| maximum service per node
    model.required_service_param = Param(model.nodes, initialize=lambda model, i: abs(model.net_flow_param[i]))

    model.bus_capacity = Param(model.trucks, initialize=[MAX_BUS_CAPACITY for _ in model.trucks])

# Descision Variables

In [13]:
def model_variables(model):
    # xijk , i!=j
    # Only create variables for i != j to avoid (A, A, bus) duplicates
    model.x = Var(
        [(i, j, k) for i in model.nodes for j in model.nodes for k in model.trucks if i != j],
        domain=Binary
    )
    
    model.u = Var(model.nodes, model.trucks, domain=NonNegativeIntegers)  # For subtour elimination
    model.z = Var(model.nodes, model.trucks, domain=NonNegativeReals)  # Service delivered to node
    model.y = Var(model.nodes, model.trucks, domain=NonNegativeReals)  # Load on truck at node
    model.w = Var(model.trucks, domain=Binary)  # Is a truck used or not

# Objective Function

In [14]:
def min_distance_rule(model):
    return sum(model.distance_param[i, j] * model.x[i, j, k]
               for i in model.nodes
               for j in model.nodes
               for k in model.trucks
               if i != j)

# Constraints

### Truck Departure/Return

In [15]:
def truck_departure_rule(model, k):
    return sum(model.x['Warehouse', j, k] for j in model.nodes if j != 'Warehouse') == model.w[k]
def truck_return_rule(model, k):
    return sum(model.x[i, 'Warehouse', k] for i in model.nodes if i != 'Warehouse') == model.w[k]

# model.truck_departure_constr = Constraint(model.trucks, rule=truck_departure_rule)
# model.truck_return_constr = Constraint(model.trucks, rule=truck_return_rule)

### Flow Constraints

In [16]:
def flow_conservation_rule(model, n, k): #n in model.consumer_nodes
    if n == 'Warehouse':
        return Constraint.Skip
    return (sum(model.x[i, n, k] for i in model.nodes if i != n) ==
            sum(model.x[n, j, k] for j in model.nodes if j != n))

## Split Service Constraints

### Service Demand Fullfillment

In [17]:
def demand_fullfillment_rule(model, n): # n in model.consumer_nodes
    return sum(model.z[n, k] for k in model.trucks) == model.required_service_param[n]

### Route to Service Constraint

In [18]:
def route_service_rule(model, n, k): #i in model.consumer_nodes
    return sum(model.x[j,n,k] * M for j in model.nodes if j != n) >= (model.z[n,k] )

### Service Limit

In [19]:
def service_limit_rule(model, i, k): #i in model.consumer_nodes : k in model.trucks
    return model.z[i,k] <= model.required_service_param[i]

## Loading and Capacity Constraints

### Initial/Final Load 

In [20]:
def depot_capacity(model,k):
    return model.y['Warehouse',k] == 0


### Load Capacity

In [21]:
def load_capacity_rule(model, i, k): # i in model.nodes, k in model.trucks
    return model.y[i,k] <= model.bus_capacity[k]

def non_negative_load_rule(model, i, k): # i in model.nodes, k in model.trucks
    return model.y[i,k] >= 0

### Load Flow Conservation

In [22]:
def get_serviceIK(model, i, k):
    if i in model.supply_nodes:
        service_ik = model.z[i,k]
    elif i in model.demand_nodes:
        service_ik = -model.z[i,k]
    else:
        service_ik = 0
    
    return service_ik

def loading_upper_bound_rule(model, i, j, k): # i in model.nodes, j in model.nodes, k in model.trucks
    if i == j:
        return Constraint.Skip
    
    service_ik = get_serviceIK(model, i, k)
    return model.y[i,k] + service_ik - model.y[j,k] <= M * (1 - model.x[i,j,k])

def loading_lower_bound_rule(model, i, j, k):
    if i == j:
        return Constraint.Skip
    
    service_ik = get_serviceIK(model, i, k)
    
    return model.y[i,k] + service_ik - model.y[j,k] >= -1 * M * (1 - model.x[i,j,k])
    
    

### Load Feasibility Check

In [23]:
def pickup_feasibility_rule(model, i, k): # i in model.supply_nodes
    return model.y[i,k] + model.z[i,k] <= model.bus_capacity[k] + M * (1 - sum(model.x[j,i,k] for j in model.nodes if j != i))

def dropoff_feasibility_rule(model, i, k): # i in model.demand_nodes
    return model.y[i,k] >= model.z[i,k] - M * (1 - sum(model.x[j,i,k] for j in model.nodes if j != i))

## Subtour Constraints

In [24]:
def mtz_initialization_rule(model, k):
    return model.u['Warehouse',k] == 0

def mtz_flow_linkage_rule(model, i, j, k): #i in model.consumer_nodes : j in model.consumer_nodes : k in model.trucks
    if i == j:
        return Constraint.Skip
    n = len(model.consumer_nodes)
    return model.u[i,k] - model.u[j,k] + (n) * model.x[i,j,k] <= n - 1

def mtz_bounds_rule_lower(model, i, k): #i in model.consumer_nodes
    return model.u[i,k] >= 1

def mtz_bounds_rule_upper(model, i, k): #i in model.consumer_nodes
    n = len(model.consumer_nodes)
    return model.u[i,k] <= n


# Build Model

In [25]:
def build_model(model, optim='time'):
    clear_model(model)

    model_sets(model)
    model_params(model)
    model_variables(model)
    
    # Objective
    if optim == 'time': 
        # MODIFIED: Minimize Makespan (Max time of any single bus)
        # This forces the solver to parallelize routes to finish earlier
        model.time_span = Var(domain=NonNegativeReals)
        
        # Constraint: Makespan must be >= the duration of every individual truck
        def makespan_rule(model, k):
            return model.time_span >= sum(model.duration_param[i, j] * model.x[i, j, k]
               for i in model.nodes
               for j in model.nodes
               if i != j)
        
        model.makespan_constr = Constraint(model.trucks, rule=makespan_rule)
        
        # Objective is to minimize the longest route
        model.obj = Objective(expr=model.time_span, sense=minimize)
    
    elif optim == 'distance':
        model.obj = Objective(rule=min_distance_rule, sense=minimize)
    else:
        raise ValueError("optim must be time or distance")

    # Constraints
    # Truck departure and return constraints
    model.truck_departure_constr = Constraint(model.trucks, rule=truck_departure_rule)
    model.truck_return_constr = Constraint(model.trucks, rule=truck_return_rule)

    print("\nTRUCK CONSTRAINTS")
    print("---------------")
    print(f"Truck Departure Constraint: {model.truck_departure_constr.__len__()}")
    print(f"Truck Return Constraint: {model.truck_return_constr.__len__()}")

    # Flow conservation and service constraints
    model.flow_conservation_constr = Constraint(model.consumer_nodes, model.trucks, rule=flow_conservation_rule)
    print("\n FLOW CONSERVATION CONSTRAINTS")
    print("---------------")
    print(f"Flow Conservation Constraint: {model.flow_conservation_constr.__len__()}")

    # Split Service Constraints
    model.demand_fullfillment_constr = Constraint(model.consumer_nodes, rule=demand_fullfillment_rule)
    # Route to service linkage constraints
    model.route_service_constr = Constraint(model.consumer_nodes, model.trucks, rule=route_service_rule)
    # Service Limit Constraints
    model.service_limit_constr = Constraint(model.consumer_nodes, model.trucks, rule=service_limit_rule)
    print("\n SERVICE CONSTRAINTS")
    print("---------------")
    print(f"Demand Fullfillment Constraint: {model.demand_fullfillment_constr.__len__()}")
    print(f"Route to Service Linkage Constraint: {model.route_service_constr.__len__()}")
    print(f"Service Limit Constraint: {model.service_limit_constr.__len__()}")

    # Load and capacity constraints
    model.depot_capacity_constr = Constraint(model.trucks, rule=depot_capacity)
    model.load_capacity_constr = Constraint(model.nodes, model.trucks, rule=load_capacity_rule)
    model.non_negative_load_constr = Constraint(model.nodes, model.trucks, rule=non_negative_load_rule)
    print("\n LOAD CONSTRAINTS")
    print("---------------")
    print(f"Depot Capacity Constraint: {model.depot_capacity_constr.__len__()}")
    print(f"Load Capacity Constraint: {model.load_capacity_constr.__len__()}")
    print(f"Non-Negative Load Constraint: {model.non_negative_load_constr.__len__()}")

    # Loading constraints
    model.loading_upper_bound_constr = Constraint(model.nodes, model.nodes, model.trucks, rule=loading_upper_bound_rule)
    model.loading_lower_bound_constr = Constraint(model.nodes, model.nodes, model.trucks, rule=loading_lower_bound_rule)
    print("\n LOADING CONSTRAINTS")
    print("---------------")
    print(f"Loading Upper Bound Constraint: {model.loading_upper_bound_constr.__len__()}")
    print(f"Loading Lower Bound Constraint: {model.loading_lower_bound_constr.__len__()}")


    # Pickup and Dropoff feasibility constraints
    model.pickup_feasibility_constr = Constraint(model.supply_nodes, model.trucks, rule=pickup_feasibility_rule)
    model.dropoff_feasibility_constr = Constraint(model.demand_nodes, model.trucks, rule=dropoff_feasibility_rule)
    print("\n PICKUP AND DROPOFF CONSTRAINTS")
    print("---------------")
    print(f"Pickup Feasibility Constraint: {model.pickup_feasibility_constr.__len__()}")
    print(f"Dropoff Feasibility Constraint: {model.dropoff_feasibility_constr.__len__()}")


    # MTZ Subtour elimination constraints
    model.mtz_initialization_constr = Constraint(model.trucks, rule=mtz_initialization_rule)
    model.mtz_flow_linkage_constr = Constraint(model.consumer_nodes, model.consumer_nodes, model.trucks, rule=mtz_flow_linkage_rule)
    model.mtz_bounds_lower_constr = Constraint(model.consumer_nodes, model.trucks, rule=mtz_bounds_rule_lower)
    model.mtz_bounds_upper_constr = Constraint(model.consumer_nodes, model.trucks, rule=mtz_bounds_rule_upper)
    print("\n MTZ SUBTOUR ELIMINATION CONSTRAINTS")
    print("---------------")
    print(f"MTZ Initialization Constraint: {model.mtz_initialization_constr.__len__()}")
    print(f"MTZ Flow Linkage Constraint: {model.mtz_flow_linkage_constr.__len__()}")
    print(f"MTZ Bounds Lower Constraint: {model.mtz_bounds_lower_constr.__len__()}")
    print(f"MTZ Bounds Upper Constraint: {model.mtz_bounds_upper_constr.__len__()}")

    return model

In [26]:
def solve_model(model, tee=True, runtime_limit=600, gap = 0.05):
    results = solver.solve(model, tee=True, options={'tmlim': runtime_limit, 'mipgap': gap})
    return results

In [27]:
model = build_model(model, optim='distance')

Total nodes: 27
Supply nodes: 11
Demand nodes: 15
Consumer nodes: 26
Distance from Warehouse to 35 Ave & 10 St: 6.7499 km

TRUCK CONSTRAINTS
---------------
Truck Departure Constraint: 4
Truck Return Constraint: 4

 FLOW CONSERVATION CONSTRAINTS
---------------
Flow Conservation Constraint: 104

 SERVICE CONSTRAINTS
---------------
Demand Fullfillment Constraint: 26
Route to Service Linkage Constraint: 104
Service Limit Constraint: 104

 LOAD CONSTRAINTS
---------------
Depot Capacity Constraint: 4
Load Capacity Constraint: 108
Non-Negative Load Constraint: 108

 LOADING CONSTRAINTS
---------------
Loading Upper Bound Constraint: 2808
Loading Lower Bound Constraint: 2808

 PICKUP AND DROPOFF CONSTRAINTS
---------------
Pickup Feasibility Constraint: 44
Dropoff Feasibility Constraint: 60

 MTZ SUBTOUR ELIMINATION CONSTRAINTS
---------------
MTZ Initialization Constraint: 4
MTZ Flow Linkage Constraint: 2600
MTZ Bounds Lower Constraint: 104
MTZ Bounds Upper Constraint: 104


In [28]:
sol = solve_model(model, tee=True, runtime_limit=3600*10, gap=0.1)


GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --tmlim 36000 --mipgap 0.1 --write C:\Users\matte\AppData\Local\Temp\tmpp932ol5y.glpk.raw
 --wglp C:\Users\matte\AppData\Local\Temp\tmpojvmpb8j.glpk.glp --cpxlp C:\Users\matte\AppData\Local\Temp\tmpe5f5k_oz.pyomo.lp
Reading problem data from 'C:\Users\matte\AppData\Local\Temp\tmpe5f5k_oz.pyomo.lp'...
C:\Users\matte\AppData\Local\Temp\tmpe5f5k_oz.pyomo.lp:75393: warning: lower bound of variable 'x2' redefined
C:\Users\matte\AppData\Local\Temp\tmpe5f5k_oz.pyomo.lp:75393: warning: upper bound of variable 'x2' redefined
9098 rows, 3132 columns, 42040 non-zeros
2920 integer variables, 2812 of which are binary
78205 lines were read
Writing problem data to 'C:\Users\matte\AppData\Local\Temp\tmpojvmpb8j.glpk.glp'...
66379 lines were written
GLPK Integer Optimizer 5.0
9098 rows, 3132 columns, 42040 non-zeros
2920 integer variables, 2812 of which are binary
Preprocessing...
8172 constraint coefficient(s) were reduced
841

# Save results

In [34]:
def save_results(save_path = "../results/solutionV2.csv", model = None):
    rows = []
    for key in model.x.keys():
        i, j, bus = key
        desc = model.x[key].value
        rows.append({
            "bus": "bus_" + str(bus),
            "from": i,
            "to": j,
            "value": desc,
            "load": model.y[i,bus].value,
            "load after": model.y[j,bus].value
        })

    df = pd.DataFrame(rows)
    df.to_csv(save_path, index=False)
    print(f"Results saved to {save_path}")

    return df

In [30]:
results_df = save_results(save_path = SOLUTION_DIR, model = model)
results_df.head(20)

Results saved to ../results/solutionV2_test2_distance.csv


,bus,from,to,value,load,load after
0,bus_0,Stagg St & Union Ave,35 Ave & 10 St,0.0,0.0,0.0
1,bus_1,Stagg St & Union Ave,35 Ave & 10 St,0.0,0.0,0.0
2,bus_2,Stagg St & Union Ave,35 Ave & 10 St,0.0,0.0,0.0
3,bus_3,Stagg St & Union Ave,35 Ave & 10 St,0.0,21.0,15.0
4,bus_0,Stagg St & Union Ave,36 Ave & 10 St,0.0,0.0,0.0
5,bus_1,Stagg St & Union Ave,36 Ave & 10 St,0.0,0.0,0.0
6,bus_2,Stagg St & Union Ave,36 Ave & 10 St,0.0,0.0,0.0
7,bus_3,Stagg St & Union Ave,36 Ave & 10 St,0.0,21.0,14.0
8,bus_0,Stagg St & Union Ave,14 St & 5 Ave,0.0,0.0,0.0
9,bus_1,Stagg St & Union Ave,14 St & 5 Ave,0.0,0.0,0.0


# View Results

In [35]:
def view_results(model, df):
    # View Results
    print("Objective Value: ", model.obj())
    
    for bus in range(N_BUSES):
        print(f"\n{'='*80}")
        print(f"ROUTE FOR BUS_{bus}")
        print(f"{'='*80}\n")

        df_bus = df[df['bus'] == f'bus_{bus}']
        df_bus = df_bus[df_bus['value'] == 1]
        
        if len(df_bus) == 0:
            print(f"Bus_{bus} is not used.\n")
            continue

        # Get all stations visited by this bus
        df_bus_stations = df_bus['from'].unique()

        route_num = 1
        start = 'Warehouse'
        visited_stations = set()
        
        while len(df_bus_stations) > 0:
            sub_route = [start]
            sub_load = [0]  # Start with 0 load
            
            # Remove start from unvisited stations
            df_bus_stations = df_bus_stations[df_bus_stations != start]
            visited_stations.add(start)
            
            current_node = None
            while current_node != start:
                next_nodes = df_bus[df_bus['from'] == sub_route[-1]]['to'].values
                if len(next_nodes) == 0:
                    break
                
                current_node = next_nodes[0]
                sub_route.append(current_node)
                
                # Get load upon arrival at current_node
                current_load = model.y[current_node, bus].value
                sub_load.append(current_load)

                # Mark as visited
                if current_node in df_bus_stations:
                    df_bus_stations = df_bus_stations[df_bus_stations != current_node]
                    visited_stations.add(current_node)
            
            # Display the route with detailed load information
            print(f"Sub-route {route_num}:")
            print("-" * 80)
            
            for idx, (station, load) in enumerate(zip(sub_route, sub_load)):
                # Get service amount if not warehouse
                service = 0
                if station != 'Warehouse' and station in model.consumer_nodes:
                    service = model.z[station, bus].value
                    service_type = "pickup" if station in model.supply_nodes else "delivery"
                else:
                    service_type = "none"
                
                # Calculate load after service
                if station in model.supply_nodes:
                    load_after = load + service
                elif station in model.demand_nodes:
                    load_after = load - service
                else:
                    load_after = load
                
                # Print station info
                arrow = " -> " if idx < len(sub_route) - 1 else ""
                print(f"  {station:30s} | Load on arrival: {load:6.2f} | "
                      f"Service: {service:6.2f} ({service_type:8s}) | "
                      f"Load after: {load_after:6.2f}{arrow}")
            
            print()
            route_num += 1
            
            # Break if no more stations to visit
            if len(df_bus_stations) == 0:
                break
            
            # Start next sub-route from an unvisited station
            start = df_bus_stations[0]

    # Summary statistics
    visited_paths = df[df['value'] == 1][['from', 'to']]
    print(f"\n{'='*80}")
    print("SUMMARY")
    print(f"{'='*80}")
    print(f"Total unique nodes visited: {len(visited_paths['from'].unique())}")
    print(f"Total edges used: {len(visited_paths)}")
    
    return visited_paths

# Call the function
view_results(model, pd.read_csv(SOLUTION_DIR))
print("Number of nodes to visit:", len(model.nodes))

Objective Value:  89.30109999999999

ROUTE FOR BUS_0

Bus_0 is not used.


ROUTE FOR BUS_1

Bus_1 is not used.


ROUTE FOR BUS_2

Sub-route 1:
--------------------------------------------------------------------------------
  Warehouse                      | Load on arrival:   0.00 | Service:   0.00 (none    ) | Load after:   0.00 -> 
  W 35 St & 9 Ave                | Load on arrival:   0.00 | Service:   5.00 (pickup  ) | Load after:   5.00 -> 
  W 43 St & 10 Ave               | Load on arrival:   5.00 | Service:   5.00 (delivery) | Load after:   0.00 -> 
  Warehouse                      | Load on arrival:   0.00 | Service:   0.00 (none    ) | Load after:   0.00


ROUTE FOR BUS_3

Sub-route 1:
--------------------------------------------------------------------------------
  Warehouse                      | Load on arrival:   0.00 | Service:   0.00 (none    ) | Load after:   0.00 -> 
  E 33 St & 5 Ave                | Load on arrival:   0.00 | Service:  12.00 (pickup  ) | Load after: 

In [36]:
from street_view import read_data, draw_map
some_stations, warehouse_stations, consumer_stations, supplier_stations, path_coords = read_data(SOME_STATIONS_DIR,
          SOLUTION_DIR)
map = draw_map(some_stations, warehouse_stations,
                consumer_stations, supplier_stations, path_coords )


ROUTE VALIDATION

bus_0: Not used (0 edges)

bus_1: Not used (0 edges)

bus_2:
  Total edges in solution: 3
  Stations in reconstructed route: 4
  Expected edges from route: 3
  Route: Warehouse → W 35 St & 9 Ave → W 43 St & 10 Ave → Warehouse

bus_3:
  Total edges in solution: 26
  Stations in reconstructed route: 27
  Expected edges from route: 26
  Route: Warehouse → E 33 St & 5 Ave → W 35 St & 9 Ave → E 15 St & 5 Ave → 6 Ave & Broome St → Lispenard St & Broadway → St Marks Pl & 2 Ave → Ave D & E 3 St → Bialystoker Pl & Delancey St → Stagg St & Union Ave → Adelphi St & Myrtle Ave → 3 Ave & Schermerhorn St → Montague St & Clinton St → Degraw St & Hoyt St → 14 St & 5 Ave → Carroll St & Rochester Ave → Broadway & 74 St → 87 St & 30 Ave → 35 Ave & 10 St → 36 Ave & 10 St → 34 Ave & 31 St → 47 Ave & Skillman Ave → 44 St & 48 Ave → Woodside Ave & 55 St → Columbus Ave & W 103 St → W 35 St & 8 Ave → Warehouse

Average location (lat, long): [np.float64(40.730579187760675), np.float64(-73.963

In [37]:
map

# Results

Our job is complete, we have sucessfully routed the trucks to follow the most optimal path for both **least time** and **least distance**. Because we do not yet have access to the supercomputers available in *New York* our tests have been limited but successfull. We have run the following test scenarios:

### TEST 1
This test consited of a smaller batch, we considered much fewer stations than possible such to confirm that a feasible solution is possible.

<center> 9 Consumer Stations, 1 Warehourse, 2 Buses </center>

The test was run to solve on **optimal time** and **optimal distance** and the results were saved and can be *roughly* viewd by changing the following and running the above map displayer.
* `SOME_STATIONS_DIR = "../processed_data/some_stations_T1.csv"`
* `SOLUTION_DIR = "../results/solutionV2_test1_distance.csv"` or `"../results/solutionV2_test1_time.csv"`

The computational time for **least distance** was near instant as the search space is much more limited. However, the method struggles significantly with the **least time** solution. The branch and bound algorithm does guarantee optimality through effectively searching through the entire space. This of course is very slow for a problem as big and complex as this one. As previously described, the MTX constraints are generally the problem because of how they scale. The number of constraints is so big for such a small problem with just 9 consumer stations.

* TRUCK CONSTRAINTS
    * Truck Departure Constraint: 2
    * Truck Return Constraint: 2

* FLOW CONSERVATION CONSTRAINTS

    * Flow Conservation Constraint: 18

* SERVICE CONSTRAINTS
    * Demand Fullfillment Constraint: 9
    * Route to Service Linkage Constraint: 18
    * Service Limit Constraint: 18

* LOAD CONSTRAINTS
    * Depot Capacity Constraint: 2
    * Load Capacity Constraint: 20
    * Non-Negative Load Constraint: 20

* LOADING CONSTRAINTS

    * Loading Upper Bound Constraint: 180
    * Loading Lower Bound Constraint: 180

* PICKUP AND DROPOFF CONSTRAINTS

    * Pickup Feasibility Constraint: 8
    * Dropoff Feasibility Constraint: 10

* MTZ SUBTOUR ELIMINATION CONSTRAINTS

    * MTZ Initialization Constraint: 2
    * MTZ Flow Linkage Constraint: 144
    * MTZ Bounds Lower Constraint: 18
    * MTZ Bounds Upper Constraint: 18

You can view the results of the **time** trial in `../results/street_view_Test1_time.html`. The results came withing 11% of the predicted optimal value after branching for 10hours

You can view the results of the **time** trial in `../results/street_view_Test1_dist.html`. The results converged to the global optimum


### TEST 2
This test consited of a larger batch of data. We considered a much larger solution space by taking more consumer nodes and increasing the number of buses.

<center>27 Consumer Stations, 1 Warehouse, 4 Buses</center>

The test was run to solve on **optimal time** and **optimal distance** and the results were saved and can be *roughly* viewd by changing the following and running the above map displayer.
* `SOME_STATIONS_DIR = "../processed_data/some_stations_T2.csv"`
* `SOLUTION_DIR = "../results/solutionV2_test2_distance.csv"` or `"../results/solutionV2_test2_time.csv"`

Total nodes: 27
Supply nodes: 11
Demand nodes: 15
Consumer nodes: 
Total Buses : 4


* TRUCK CONSTRAINTS

    * Truck Departure Constraint: 4
    * Truck Return Constraint: 4

* FLOW CONSERVATION CONSTRAINTS

    * Flow Conservation Constraint: 104

* SERVICE CONSTRAINTS

    * Demand Fullfillment Constraint: 26
    * Route to Service Linkage Constraint: 104
    * Service Limit Constraint: 104

* LOAD CONSTRAINTS

    * Depot Capacity Constraint: 4
    * Load Capacity Constraint: 108
    * Non-Negative Load Constraint: 108

* LOADING CONSTRAINTS

    * Loading Upper Bound Constraint: 2808
    * Loading Lower Bound Constraint: 2808

* PICKUP AND DROPOFF CONSTRAINTS

    * Pickup Feasibility Constraint: 44
    * Dropoff Feasibility Constraint: 60

* MTZ SUBTOUR ELIMINATION CONSTRAINTS

    * MTZ Initialization Constraint: 4
    * MTZ Flow Linkage Constraint: 2600
    * MTZ Bounds Lower Constraint: 104
    * MTZ Bounds Upper Constraint: 104

You can view the results of the **time** trial in `../results/street_view_Test2_time.html`. The results came withing 11% of the predicted optimal value

You can view the results of the **time** trial in `../results/street_view_Test2_dist.html`. The results came within 22% of the predicted optimal value after 10 hours of branching.

# APPENDIX

To justify this set of constraints, lets think of this constraint in plain english. 
$$ \text{Load Arriving at } i \pm \text{Service at } i = \text{Load Arriving at } j$$
$$ \text{or} $$
$$ \text{Load After leaving } i = \text{Load Arriving at } j $$

But this is not as simple as just a single equality, rather we can use Big-M constraints to either relax or tighten the constraint. To explain this consider the two cases:
* No bus ($x_{ijk} = 0$): 
    $$ \text{Load After Leaving } i - \text{Load Arriving at } j \in \mathbb{R} $$
    Given we dont travel $i \rightarrow j$ directly, there is no connection between what happens at these stations. We may arrive at $j$ later through another node or ,by chance, the load at $j$ may be the same. So in theory the difference between the load **after leaving** $i$ and **upon arriving** $j$ can be any number. Of course out computer dosent like that so we can use $M$ which is just a large number to formulate it as:
    $$ -M \leq \text{Load After leaving } i = \text{Load Arriving at } j \leq M $$

* Bus ($x_{ijk} = 1$)
    $$ \text{Load After Leaving } i - \text{Load Arriving at } = 0$$
    Conversely, if we go from $i \rightarrow j$ then we expect the load *after leaving* $i$ to be the same as the load *arriving* at $j$

    The final step is to compute the service done based on if its a supply node ($>0$), a demand node ($<0$) or the warehouse ($=0$) 